In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Metals/P1011.2. Analyses/P1011.2.2. Metal Prediction/Output')
cur_date = "040524"

library(readxl)
library(openxlsx)
library(writexl)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_020924.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.3     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.3     ✔ tibble  3.2.1
✔ purrr   1.0.2     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,Ar,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816,2.007062,ND
2,04009002C,1525,14-18,12/7/10,65,300,32,2.0,7.0,Ar,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912,1.912321,ND
3,04030015C,1525,14-18,8/27/12,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,7.000000,D
4,04030015J,1525,14-18,4/5/10,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,1.153921,ND
5,04030020H,234,09-147,10/25/10,52,125,36,20.0,7.6,Ar,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,1.675360,ND
6,4030041,1515,14-04,3/2/16,47,275,34,2.5,8.2,Ar,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955,14.000000,D


# Prediction of Contaminated Wells using Supervised ML

Using `Water_Sample_Date`, `Casing_Depth`, `Well_Depth`, `Static_Water_Depth`, `pH`, `Flow_Rate`, `Stream_Distance`, `Elevation`, `Geologic_Terrane` or `Rock_Type`, `Soil_Type_Condensed`, `Landuse_Condensed`, `Latitude`, and `Longitude` to predict whether Mn concentration falls above or below the EPA's Maximum Contaminant Level (MCL) (< or >= 10 ppb). RF and SVM models will be built to predict concentration this binarized concentration. 

Additionally, models will be run using a different combination of the aforementioned predictors detailed into the following use cases below:

1. All Data (using all 13 features)
2. All variables excluding latitude and longitude (Using this to see if the prediction's accuracy is retained removing those coordinates to make the results more generalizable to areas outside of Union County, NC.)
3. Well Chacteristics only (casing depth, pH, flow rate, well depth, and static water depth)
4. Health Department lens (rock type, latitude, and longitude)

Starting by creating an additional variable for above and below 10 ppb and formatting the df for input into ML models.

In [3]:
arsenic_data = well_data %>%
    mutate(MCL = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date)) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar" & Landuse_Condensed != "NA") %>%
    select(-Metal)

# changing data types
arsenic_data$Geologic_Terrane = factor(arsenic_data$Geologic_Terrane)
arsenic_data$Rock_Type = factor(arsenic_data$Rock_Type)
arsenic_data$Soil_Type_Condensed = factor(arsenic_data$Soil_Type_Condensed)
arsenic_data$Landuse_Condensed = factor(arsenic_data$Landuse_Condensed)

head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  6 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration,MCL
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816,2.007062,ND,0
2,04009002C,1525,14-18,2010-12-07,65,300,32,2.0,7.0,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912,1.912321,ND,0
3,04030015C,1525,14-18,2012-08-27,65,300,32,2.0,8.1,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,7.000000,D,0
4,04030015J,1525,14-18,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,1.153921,ND,0
5,04030020H,234,09-147,2010-10-25,52,125,36,20.0,7.6,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,1.675360,ND,0
6,4030041,1515,14-04,2016-03-02,47,275,34,2.5,8.2,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955,14.000000,D,1


In [4]:
# original number of records
dim(well_data)

# records kept for analysis
dim(arsenic_data)

[1] 2145   20

[1] 714  20

# Variable Selection

To determine which variables are collinear and need to be removed, but first doing this with continuous data only to run Spearman's correlation tests. The continuous variables include: `casing_depth`, `well_depth`, `static_water_depth`, `pH`, `flow_rate`, `Elevation`, 
                    `Stream_Distance`,`latitude`, and `longitude`.

In [5]:
continuous_vars = c('Casing_Depth', 'Well_Depth', 'Static_Water_Depth', 'pH', 'Flow_Rate', 'Latitude', 'Longitude', 'Elevation', 
                    'Stream_Distance')

# creating correlation matrix
corr_matrix = rcorr(as.matrix(arsenic_data[,continuous_vars]), type = "spearman")
# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



Variable1,Variable2,Spearman Coefficient
<chr>,<fct>,<dbl>


No features passed the threshold, so none of the continuous variables are highly correlated to each other. I could use ANOVA tests for continuous and multi-categorical variable comparisons, however an ANOVA measures group differences as opposed to correlations. The categorical variables include: `Geologic_Terrane`, `Rock_Type`, `Soil_Type_Condensed`, and `Landuse_Condensed`. I ended up not doing this.

In [6]:
cateogorical_vars = c('Geology', 'Soil_Type', 'Landuse')
# t = aov(Longitude ~ Geologic_Terrane, data = arsenic_data)
# summary(t)

# aov_test = function(variable1, variable2){
#     t = aov(variable1 ~ variable2, data = arsenic_data)
#     output = summary(t)
#     return(output)
# }
# aov_test('Longitude', 'Geologic_Terrane')

Running correlations between categorical variables using a chi-square test.

In [7]:
get_tables = function(df, variable1, variable2){
    # creating tables for each variable
    overall_variable = df %>% 
        # need the group_by_at function to pass a function parameter into dplyr
        group_by_at(variable1) %>%
        select(all_of(variable2)) 

    variable_table = table(overall_variable)
    return(variable_table)
}

# calling fn
geology_soil_table = get_tables(arsenic_data, 'Geologic_Terrane', 'Soil_Type_Condensed')
geology_landuse_table = get_tables(arsenic_data, 'Geologic_Terrane', 'Landuse_Condensed')
landuse_soil_table = get_tables(arsenic_data, 'Landuse_Condensed', 'Soil_Type_Condensed')
rock_soil_table = get_tables(arsenic_data, 'Rock_Type', 'Soil_Type_Condensed')
rock_landuse_table = get_tables(arsenic_data, 'Rock_Type', 'Landuse_Condensed')

# geology_soil_table
# geology_landuse_table
# landuse_soil_table
# rock_soil_table
# rock_landuse_table

Adding missing grouping variables: `Geologic_Terrane`
Adding missing grouping variables: `Geologic_Terrane`
Adding missing grouping variables: `Landuse_Condensed`
Adding missing grouping variables: `Rock_Type`
Adding missing grouping variables: `Rock_Type`


In [8]:
# fisher exact test
fisher.test(geology_soil_table)
fisher.test(geology_landuse_table) 
# not enough data and threw errors, so I changed the simulate p value parameter
fisher.test(landuse_soil_table, simulate.p.value = TRUE)
fisher.test(rock_soil_table, simulate.p.value = TRUE)
fisher.test(rock_landuse_table, simulate.p.value = TRUE)


	Fisher's Exact Test for Count Data

data:  geology_soil_table
p-value = 4.508e-05
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  geology_landuse_table
p-value = 0.0004902
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  landuse_soil_table
p-value = 0.0004998
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  rock_soil_table
p-value = 0.0004998
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  rock_landuse_table
p-value = 0.0004998
alternative hypothesis: two.sided


Based on this, all these variables are highly correlated to each other. Moving forward, I'll excluded `Geologic_Terrane` and kept `Rock_Type`.

# Summary Statistics
Determining if there are any signficiant differences between the predictor variables for each outcome variable.

In [9]:
arsenic_data %>%
  tbl_summary(by = MCL, missing = "no", 
  include = colnames(arsenic_data[c(4:17)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
  add_n() %>% 
  #add_overall() %>%
  add_p(test = list(all_continuous() ~ "t.test",
                    all_categorical() ~ "chisq.test")) %>% # adding p value from anova
  as_tibble() #%>%
  #write_xlsx(., "Output/TableX.xlsx")

There was an error in 'add_p()/add_difference()' for variable 'Water_Sample_Date', p-value omitted:
Error in Math.Date(mx): abs not defined for "Date" objects

Warning for variable 'Rock_Type':
simpleWarning in stats::chisq.test(x = structure(c(4L, 4L, 4L, 4L, 4L, 4L, 4L, : Chi-squared approximation may be incorrect

Warning for variable 'Soil_Type_Condensed':
simpleWarning in stats::chisq.test(x = structure(c(2L, 1L, 1L, 2L, 2L, 2L, 3L, : Chi-squared approximation may be incorrect

Warning for variable 'Landuse_Condensed':
simpleWarning in stats::chisq.test(x = structure(c(1L, 1L, 1L, 3L, 3L, 2L, 1L, : Chi-squared approximation may be incorrect



**Characteristic**,**N**,"**0**, N = 582","**1**, N = 132",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,712,2013-05-13 (973.00384387001),2013-05-03 (979.450720926567),NA
Casing_Depth,714,73 (33),51 (19),<0.001
Well_Depth,714,306 (146),327 (109),0.057
Static_Water_Depth,714,35 (13),34 (11),0.4
Flow_Rate,714,24 (32),13 (15),<0.001
pH,714,7.48 (0.54),7.87 (0.41),<0.001
Longitude,714,-80.65 (0.13),-80.49 (0.11),<0.001
Latitude,714,34.97 (0.09),35.02 (0.09),<0.001
Geologic_Terrane,714,NA,NA,<0.001


In [10]:
# creating dfs for each use case
# dropped 2 rows that had missing dates
arsenic_case_1_df = drop_na(arsenic_data[,c(4:11,13:17,20)])
arsenic_case_2_df = drop_na(arsenic_data[,c(4:9,13:17,20)])
arsenic_case_3_df = drop_na(arsenic_data[,c(5:9,20)])
arsenic_case_4_df = drop_na(arsenic_data[,c(10,11,13,20)])

head(arsenic_case_1_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,MCL
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>
1,2010-12-07,65,300,32,2.0,7.0,-80.55676,34.87109,MV,D,C,678.113,454.1912,0
2,2012-08-27,65,300,32,2.0,8.1,-80.55676,34.87109,MV,D,C,469.708,454.1912,0
3,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,MV,H,F,470.293,454.1912,0
4,2010-10-25,52,125,36,20.0,7.6,-80.55220,34.86012,MV,H,F,470.293,918.3419,0
5,2016-03-02,47,275,34,2.5,8.2,-80.56423,34.88559,MV,H,D,470.293,512.3955,1
6,2009-07-14,36,200,23,25.0,8.3,-80.58317,34.87103,MV,K,C,507.966,158.4799,0


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [11]:
rf_classification = function(dataset, outcome, pred_outcome, use_case){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, lower CI, upper CI, sens, spec, PPV, NPV to take mean later
        matrix_values = data.frame(t(c(matrix$overall[c(1,3,4)])), t(c(matrix$byClass[11])), 
                                   t(c(matrix$byClass[1:4])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(Accuracy = mean(Accuracy), LowerCI = mean(AccuracyLower), UpperCI = mean(AccuracyUpper),
                  Sensitivity = mean(Sensitivity), Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), 
                  NPV = mean(Neg.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini) %>%
        mutate(Use_Case = use_case)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [12]:
# calling fn
rf_values_arsenic_case_1 = rf_classification(arsenic_case_1_df, "MCL", "pred_MCL",1)
rf_values_arsenic_case_2 = rf_classification(arsenic_case_2_df, "MCL", "pred_MCL",2)
rf_values_arsenic_case_3 = rf_classification(arsenic_case_3_df, "MCL", "pred_MCL",3)
rf_values_arsenic_case_4 = rf_classification(arsenic_case_4_df, "MCL", "pred_MCL",4)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [13]:
# viewing results
rf_confusion_matrix = data.frame(Model = "RF Classification", 
                                 Use_Case = 1:4, Kernel = NA, 
                                 rbind(rf_values_arsenic_case_1[[2]], rf_values_arsenic_case_2[[2]],
                                 rf_values_arsenic_case_3[[2]],rf_values_arsenic_case_4[[2]])) 

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_case_1[[3]]

Model,Use_Case,Kernel,Accuracy,LowerCI,UpperCI,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<int>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,1,NA,0.8371910,0.7666062,0.8931779,0.6225071,0.8862069,0.5560080,0.9114514,0.7543570
RF Classification,2,NA,0.8301684,0.7586538,0.8874096,0.5621083,0.8913793,0.5438852,0.8993424,0.7267438
RF Classification,3,NA,0.7911720,0.7160072,0.8538467,0.4384615,0.8709696,0.4534548,0.8721466,0.6547156
RF Classification,4,NA,0.8150463,0.7416623,0.8748981,0.7116809,0.8384321,0.5022757,0.9279459,0.7750565


Predictor,MeanDecreaseGini,Use_Case
<chr>,<dbl>,<dbl>
Longitude,127.290950,1
Rock_Type,49.685113,1
Latitude,45.692223,1
pH,44.030740,1
Casing_Depth,41.173618,1
Well_Depth,29.212972,1
Flow_Rate,27.749159,1
Static_Water_Depth,20.501384,1
Elevation,18.279791,1


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise.

In [14]:
train_vars_noise_arsenic_case_1 = rf_values_arsenic_case_1[[1]]

noise_df = function(train_vars_noise){
    set.seed(8)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[10])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[2])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[9])]], replace = TRUE)
    
    return(train_vars_noise)
}

# calling fn
noise_training_rf_dataset_arsenic_case_1 = noise_df(train_vars_noise_arsenic_case_1)

head(noise_training_rf_dataset_arsenic_case_1)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,MCL,noise1,noise2,noise3,noise4,noise5
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>
1,2012-08-27,65,300,32,2.0,8.1,-80.55676,34.87109,MV,D,C,469.708,454.1912,0,H,53.00000,18.50846,7.700000,M
2,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,MV,H,F,470.293,454.1912,0,D,42.00000,30.00000,7.800000,MV
3,2010-10-25,52,125,36,20.0,7.6,-80.55220,34.86012,MV,H,F,470.293,918.3419,0,H,44.00000,12.21333,7.834375,MV
4,2016-03-02,47,275,34,2.5,8.2,-80.56423,34.88559,MV,H,D,470.293,512.3955,1,NA,44.00000,20.00000,7.871824,M
5,2009-07-14,36,200,23,25.0,8.3,-80.58317,34.87103,MV,K,C,507.966,158.4799,0,H,53.35205,12.00000,6.600000,MV
6,2014-07-09,60,200,30,8.0,7.6,-80.58877,34.87173,MV,H,C,542.394,377.8589,0,H,47.00000,15.00000,7.200000,M


In [15]:
# calling fn
noise_rf_values_arsenic_case_1 = rf_classification(noise_training_rf_dataset_arsenic_case_1, "MCL", 
                                             "pred_MCL",1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [16]:
# viewing results
noise_rf_confusion_matrix = data.frame(Model = "RF w/ Noise",
            noise_rf_values_arsenic_case_1[[2]])

noise_rf_confusion_matrix

Model,Accuracy,LowerCI,UpperCI,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF w/ Noise,0.8868643,0.8324413,0.9284041,0.9051192,0.8685601,0.8738844,0.9023408,0.8868396


Calculating the average noise rank to determine, which predictors ranked higher than the highest noise rank.

In [17]:
noise_importance_values_case_1 = noise_rf_values_arsenic_case_1[[3]] %>%
    column_to_rownames(var = "Predictor")

In [18]:
highest_noise_rank = function(noise_importance_values){
    # this gives us values between 0 and 1
    # the closer the number gets to 1 the lower it ranks, so the number closest to 1 is the max rank
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    # figuring out which is the highest mean decrease gini or background noise importance value
    highest_noise_rank = min(noise1rank, noise2rank, noise3rank, noise4rank, noise5rank)
    
    return(highest_noise_rank)
}

# calling fn
highest_noise_rank_case_1 = highest_noise_rank(noise_importance_values_case_1)
highest_noise_rank_case_1

[1] 0.7222222

In [19]:
last_significant_rank = function(noise_importance_values, highest_noise_rank){
    for (i in 1:length(noise_importance_values$MeanDecreaseGini)){
    # if the row number (index)/# of variables is less than the highest noise rank the loop will continue
        if (i/length(noise_importance_values$MeanDecreaseGini) > highest_noise_rank){
            last_significant_var = i - 1
            break
        }
    }    
    return(last_significant_var)
}

# calling fn
last_significant_rank_case_1 = last_significant_rank(noise_importance_values_case_1, highest_noise_rank_case_1)
last_significant_rank_case_1

[1] 13

In [20]:
# now getting those significant predictors
significant_predictors_case_1 = rownames(noise_importance_values_case_1)[1:last_significant_rank_case_1]

In [21]:
# converting row names back to columns to make next step easier
noise_importance_values_case_1 = noise_importance_values_case_1 %>%
    rownames_to_column(var = "Predictor")

In [22]:
# getting dataframe ready to export variable importance ranks from above and whether those features passed the 
# background filter
noise_variable_importance_ranks = function(noise_importance_values_df, significant_predictors, use_case){
    # Determing what variables fall above the random noise variable importance ranks
    # :parameters: a dataframe containing the initial feature importance values, last position considered to be significant,
    # outcome predicted
    # :output: a dataframe containing the feature, mean decrease gini, significance, outcome predicted, variables included 
    # in the model
    
    # filtering the original rank df for predictors that were significant (above random noise)
    filtered_significant_ranks_df = noise_importance_values_df %>%
        filter(Predictor %in% significant_predictors) %>%
        arrange(-MeanDecreaseGini)
    
    # adding a column denoting if the feature was above random noise
    filtered_significant_ranks_df$Passed_Filter = c(rep(c("Yes"), 
                                                        times = length(filtered_significant_ranks_df$Predictor)))
    
    final_df = full_join(filtered_significant_ranks_df, noise_importance_values_df)
    
    # adding a col for the predictors used
    final_df$Use_Case = use_case
    
    return(final_df)
}

# calling fn
significant_predictors_df_case_1 = noise_variable_importance_ranks(noise_importance_values_case_1, 
                                                                   significant_predictors_case_1, 1)

# creating 1 df
significant_predictors_df_3_cases = rbind(rf_values_arsenic_case_2[[3]], rf_values_arsenic_case_3[[3]],
                                 rf_values_arsenic_case_4[[3]]) %>%
    # Adding a column to match the first use case df
    mutate(Passed_Filter = NA)

significant_predictors_df = rbind(significant_predictors_df_case_1, significant_predictors_df_3_cases)

head(significant_predictors_df)

Joining with `by = join_by(Predictor, MeanDecreaseGini, Use_Case)`


,Predictor,MeanDecreaseGini,Use_Case,Passed_Filter
,<chr>,<dbl>,<dbl>,<chr>
1,Longitude,91.50081,1,Yes
2,Rock_Type,40.50729,1,Yes
3,Casing_Depth,35.09253,1,Yes
4,pH,34.35974,1,Yes
5,Latitude,30.48440,1,Yes
6,Well_Depth,18.57035,1,Yes


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [23]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values, use_case){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, lower CI, upper CI, sens, spec, PPV, NPV to take mean later
        matrix_values = data.frame(t(c(matrix$overall[c(1,3,4)])), t(c(matrix$byClass[11])), 
                                   t(c(matrix$byClass[1:4])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
    # taking averages/sd 
    metrics = metrics %>%
        summarise(Accuracy = mean(Accuracy), LowerCI = mean(AccuracyLower), UpperCI = mean(AccuracyUpper),
                  Sensitivity = mean(Sensitivity), Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), 
                  NPV = mean(Neg.Pred.Value), AUC = mean(auc)) %>%
    mutate(Model = "SVM", Use_Case = use_case, Kernel = model) 

    # reordering columns
    metrics = metrics[,c(9:11,1:8)]
  
    return(metrics)
}

In [24]:
#calling fn
svm_case_1_linear = svm_classification(arsenic_case_1_df, "linear", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),1)
svm_case_1_radial = svm_classification(arsenic_case_1_df, "radial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),1)
svm_case_1_polynomial = svm_classification(arsenic_case_1_df, "polynomial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),1)
svm_case_2_linear = svm_classification(arsenic_case_2_df, "linear", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),2)
svm_case_2_radial = svm_classification(arsenic_case_2_df, "radial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),2)
svm_case_2_polynomial = svm_classification(arsenic_case_2_df, "polynomial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),2)
svm_case_3_linear = svm_classification(arsenic_case_3_df, "linear", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),3)
svm_case_3_radial = svm_classification(arsenic_case_3_df, "radial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),3)
svm_case_3_polynomial = svm_classification(arsenic_case_3_df, "polynomial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),3)
svm_case_4_linear = svm_classification(arsenic_case_4_df, "linear", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),4)
svm_case_4_radial = svm_classification(arsenic_case_4_df, "radial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),4)
svm_case_4_polynomial = svm_classification(arsenic_case_4_df, "polynomial", "MCL", "pred_MCL",
                                            0.035, 1:5, seq(0,1,0.2),4)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [25]:
# creating final df
final_df = rbind(rf_confusion_matrix, svm_case_1_linear, svm_case_1_radial, svm_case_1_polynomial, 
                 svm_case_2_linear, svm_case_2_radial, svm_case_2_polynomial, svm_case_3_linear, 
                 svm_case_3_radial, svm_case_3_polynomial, svm_case_4_linear, svm_case_4_radial, 
                 svm_case_4_polynomial) 

final_df

Model,Use_Case,Kernel,Accuracy,LowerCI,UpperCI,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,1,NA,0.8371910,0.7666062,0.8931779,0.6225071,0.8862069,0.5560080,0.9114514,0.7543570
RF Classification,2,NA,0.8301684,0.7586538,0.8874096,0.5621083,0.8913793,0.5438852,0.8993424,0.7267438
RF Classification,3,NA,0.7911720,0.7160072,0.8538467,0.4384615,0.8709696,0.4534548,0.8721466,0.6547156
RF Classification,4,NA,0.8150463,0.7416623,0.8748981,0.7116809,0.8384321,0.5022757,0.9279459,0.7750565
SVM,1,linear,0.7922092,0.7166727,0.8551772,0.8190883,0.7862069,0.4689997,0.9501817,0.8026476
SVM,1,radial,0.8020487,0.7273190,0.8637765,0.7210826,0.8206897,0.4777414,0.9281029,0.7708861
SVM,1,polynomial,0.7641091,0.6859257,0.8309709,0.8339031,0.7482759,0.4312918,0.9518775,0.7910895
SVM,2,linear,0.7795233,0.7026641,0.8443831,0.7886040,0.7775862,0.4481283,0.9414808,0.7830951
SVM,2,radial,0.7907712,0.7147947,0.8542549,0.7205128,0.8068966,0.4593511,0.9268913,0.7637047


In [26]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "As_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(significant_predictors_df, paste0(Output,"/", "As_Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)